In [1]:
# import useful libraries

import numpy as np
import itertools
from collections import Counter
from utils import load_puzzle_input, test_code

In [2]:
# load puzzle input
    
puzzle_input = load_puzzle_input('input.txt')

print(puzzle_input[:3])

['471,144 -> 471,134 -> 471,144 -> 473,144 -> 473,140 -> 473,144 -> 475,144 -> 475,143 -> 475,144 -> 477,144 -> 477,142 -> 477,144 -> 479,144 -> 479,137 -> 479,144 -> 481,144 -> 481,134 -> 481,144 -> 483,144 -> 483,136 -> 483,144 -> 485,144 -> 485,134 -> 485,144', '475,166 -> 475,159 -> 475,166 -> 477,166 -> 477,156 -> 477,166 -> 479,166 -> 479,165 -> 479,166 -> 481,166 -> 481,156 -> 481,166 -> 483,166 -> 483,163 -> 483,166 -> 485,166 -> 485,156 -> 485,166 -> 487,166 -> 487,159 -> 487,166', '513,69 -> 513,71 -> 511,71 -> 511,76 -> 522,76 -> 522,71 -> 517,71 -> 517,69']


### Part 1

In [3]:
# code for preparing and testing examples for part 1

example_dict_part_1 = {
    tuple(load_puzzle_input('example_1.txt')): 24
}

In [4]:
def parse_puzzle_input(puzzle_input):
    
    list_of_tuple_lists = []
    rolling_tuple_list = []
    
    for line in puzzle_input:
        coordinates = [x.strip() for x in line.split('->')]
        for coordinate in coordinates:
            x, y = coordinate.split(',')
            rolling_tuple_list.append((int(x), int(y)))
        list_of_tuple_lists.append(rolling_tuple_list)
        rolling_tuple_list = []
    
    return list_of_tuple_lists

In [5]:
def generate_list_of_points_between_two_points(x1, y1, x2, y2):

    if x1 != x2:
        
        x_list = range(min(x1, x2), max(x1, x2) + 1)

        return zip(x_list, [y1] * len(x_list))

    else:
        
        y_list = range(min(y1, y2), max(y1, y2) + 1)

        return zip([x1] * len(y_list), y_list)

In [6]:
def generate_rock_structure(list_of_tuple_lists):
    
    rock_coordinate_set = set()
    
    for tuple_list in list_of_tuple_lists:
             
        for idx in range(len(tuple_list) - 1):
            
            coordinate_1, coorindate_2 = tuple_list[idx], tuple_list[idx + 1]
            x1, y1 = coordinate_1
            x2, y2 = coorindate_2
            
            for coordinate_to_add in generate_list_of_points_between_two_points(x1, y1, x2, y2):
                rock_coordinate_set.add(coordinate_to_add)
                
    return rock_coordinate_set
            

In [7]:
def find_void_y(rock_structure_set):
    """This is the y level where the sand disappears:  max y + 1"""
    
    return max(coordinate[1] for coordinate in rock_structure_set) + 1

In [8]:
def find_sand_next_move(current_sand_position, rock_structure):
    
    sand_x, sand_y = current_sand_position
    
    # downwards first
    if (sand_x, sand_y + 1) not in rock_structure:
        return (sand_x, sand_y + 1)
    
    # one step down and to the left
    elif (sand_x - 1, sand_y + 1) not in rock_structure:
        return (sand_x - 1, sand_y + 1)
    
    # one step down and to the right
    elif (sand_x + 1, sand_y + 1) not in rock_structure:
        return (sand_x + 1, sand_y + 1)
    
    # otherwise, rest
    return None

In [9]:
def find_sand_settle_point(sand_enter_point, rock_structure, void_point = None):
    
    current_sand_position = sand_enter_point
    
    while True: # this line gives me anxiety
        
        next_sand_move = find_sand_next_move(current_sand_position, rock_structure)
        
        if not next_sand_move: # i.e. rest
            return current_sand_position
        
        elif void_point:
            if next_sand_move[1] >= void_point: # i.e. fallen out of the world
                return None
        
        current_sand_position = next_sand_move

In [10]:
def part_1_solution(puzzle_input):
    
    sand_enter_point = (500, 0)

    list_of_tuple_lists = parse_puzzle_input(puzzle_input)
    rock_structure = generate_rock_structure(list_of_tuple_lists)
    void_y = find_void_y(rock_structure)

    sand_counter = 0

    while True: # anxiety
        
        if not (sand_settle_point := find_sand_settle_point(sand_enter_point, rock_structure, void_y)):
            return sand_counter
        
        sand_counter += 1
        rock_structure.add(sand_settle_point)


In [11]:
# test part 1 solution

test_code(part_1_solution, example_dict_part_1)

Test 0 passed: Input <('498,4 -> 498,6 -> 496,6', '503,4 -> 502,4 -> 502,9 -> 494,9')> gives output <24>.

Congratulations! Looks like you cracked it! Good job!


In [12]:
part_1_solution(puzzle_input)

1003

### Part 2

In [13]:
# code for preparing and testing examples for part 2

example_dict_part_2 = {
    tuple(load_puzzle_input('example_1.txt')): 93
}

In [17]:
def find_floor_line(sand_enter_point, rock_structure):
        
    floor_y = max(coordinate[1] for coordinate in rock_structure) + 2
    
    cave_height = floor_y - sand_enter_point[1]
    
    floor_x = range(sand_enter_point[0] - cave_height, sand_enter_point[0] + cave_height + 1)
    
    return zip(floor_x, [floor_y] * len(floor_x))

In [20]:
def part_2_solution(puzzle_input):
    
    sand_enter_point = (500, 0)

    list_of_tuple_lists = parse_puzzle_input(puzzle_input)
    rock_structure = generate_rock_structure(list_of_tuple_lists)
    
    for floor_point in find_floor_line(sand_enter_point, rock_structure):
        rock_structure.add(floor_point)

    sand_counter = 0

    while True: # anxiety
        
        sand_settle_point = find_sand_settle_point(sand_enter_point, rock_structure)
        
        if sand_settle_point == sand_enter_point:
            return sand_counter + 1
        
        sand_counter += 1
        rock_structure.add(sand_settle_point)


In [21]:
# test part 2 solution

test_code(part_2_solution, example_dict_part_2)

Test 0 passed: Input <('498,4 -> 498,6 -> 496,6', '503,4 -> 502,4 -> 502,9 -> 494,9')> gives output <93>.

Congratulations! Looks like you cracked it! Good job!


In [22]:
part_2_solution(puzzle_input)

25771